# FieldTools Tutorial

---
#### In this tutorial, you will calculate the active site fields in the b-lactamase KPC from a test trajectory.
---
**To that end, you will:**
1. Calculate Fields with FieldTools for an example trajectory.
2. Average & Plot Fields.
---
**Fields are calculated from MD trajectories and saved into a pickle file with the following data structure:**

    Fields[field_target][field_component][field_1, field_2, ... field_n]
    
        field_target                    : describe the location at which the field is calculated.
                                          either along a bond or at a specific atom
                                          
        field_component                 : describe the part of the system that causes the field. Components are:
                Total                   : Total field
                Protein                 : Protein field
                Solvent                 : Solvent field
                RSN_1, RSN_2, ... RSN_i : Field of each protein residue
                WAT, Na+, Cl-, etc.     : Field from each type of solvent molecule in the system
                
        field_1, field_2, ... field_n   : List of the fields in each frame of the simulation
---     
<font color="red">**Attention: Trajectories must be immaged!**</font>
    
- FieldTools calculates the fields from the exact location of all atoms in the simulation without considiring periodicity.

---

# 1. Calculate Fields with FieldTools

**FieldTools requires the following input:**

- **Trajectory (-nc) and parameter (-parm) input file**:
  - The script uses pytraj to load the trajectory and thus accepts any file types accepted by pytraj.
  
- **A -target file that specifies the positions at which the field is to be calculated**:
  - FieldTools can calculate the field either along a bond or at a specific atom.
  - For field calculations along bonds, two target atoms need to be defined.
  - For field calculations at a specific point, only one target atom needs to be defined.
  - Multiple targets can be specified and calculated in parallel.

- **A -solvent flag that specifies all solvent (and ion!) molecules**:
  - For the protein, FieldTools will give the field of each individual protein residue.
  - Solvent atoms will not be included in the protein field, but summed up separately in the solvent field.
  - The script will furthermore provide separate fields for each type of solvent molecule.

- **Optional -exclude_atoms flag that defines which atoms will not be included in the charge calculation**:
  - Some Atoms need to be excluded for the field calculations.
  - By default, these are all atoms of the residue for which the field is calculated on.
  - Alternatively, these atoms can be defined using -exclude_atoms.

- **Optional QM charges can be loaded for the calculation of the field**:
  - QM charges are calculated with a separate script (QMChargesTools).
  - If QM charges are to be loaded, the output from QMChargesTools should be used (-qm_mask, -qm_charges, and -qm_dict).
  - This is still experimental and will be added in the future!
  - Contact [adrian.bunzel@bsse.ethz.ch](mailto:adrian.bunzel@bsse.ethz.ch) for early access.
---
**The final fields will be saved into a .pkl file specified with -out.**

In [ ]:
#-- Uncomment to read FieldTools help file

#!python utils/FieldTools.py -help 

In [ ]:
#-- Prepare the input files

# Defines the position at which the field is to be calculated. Either at a point or along a bond
field_target = """:40@C7 :40@O71
:40@O71"""
!echo "$field_target" > data/field_target.dat

# Exlude atoms (needs to be repeated for every target, here :40@C7 :40@O71 and :40@O71!)
exclude_mask = """:40@HB2 :40@HB1 :40@CB :40@OG :40@C7 :40@O71 :40@C25 :40@H6 :40@CA :40@HA :264@O :264@H1 :264@H2
:40@HB2 :40@HB1 :40@CB :40@OG :40@C7 :40@O71 :40@C25 :40@H6 :40@CA :40@HA :264@O :264@H1 :264@H2"""
!echo "$exclude_mask" > data/exclude_mask.dat

In [ ]:
#-- Run the field calculations
!python utils/FieldTools.py -nc data/KPC.nc \
                            -parm data/KPC.parm7 \
                            -out data/KPC_field.pkl \
                            -exclude_atoms data/exclude_mask.dat \
                            -target data/field_target.dat \
                            -solvent WAT,Na+ \
                            -TIP4P True

# 2. Average & Plot Fields

In [ ]:
#-- Load the calculated fields
import pickle
FIELDS = pickle.load(open("data/KPC_field.pkl","rb"))

#-- print out the field keys to give an idea about the data structure
print(FIELDS.keys())
print(FIELDS[':40@C7_:40@O71'].keys())
print(FIELDS[':40@C7_:40@O71']['Total'])

In [ ]:
#-- plot the fields as bar plot
import matplotlib.pyplot as plt 
import numpy as np
plt.rcParams["figure.figsize"]=[15,2]
for Target in FIELDS:
    
    # Only protein residues, no Total, Solvent, etc.
    x_keys = [i for i in list(FIELDS[Target]) if len(i.split("_"))==2] 
    
    #get x and y data
    y = np.array([np.mean(FIELDS[Target][x_key]) for x_key in x_keys])
    x = np.array([int(x_key.split("_")[-1]) for x_key in x_keys])
    
    #plot data
    plt.bar(x, y)
    plt.axhline(0,c= "k")
    plt.title(f"Target: {Target}")
    plt.xlabel("Residue Number")
    plt.ylabel("Field (MV/cm)")
    plt.xlim(0, len(x) + 1)
    plt.ylim(-max(abs(y))*1.1, max(abs(y))*1.1)
    plt.show()